# Trying to train UNet with L1 loss

In [1]:
!pip install scikit-image
!pip install IProgress
!pip install jupyter
!pip install ipywidgets widgetsnbextension pandas-profiling
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/cytomine-uliege/Cytomine-python-client/pypi/simple
^C
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/cytomine-uliege/Cytomine-python-client/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/cytomine-uliege/Cytomine-python-client/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/cytomine-uliege/Cytomine-python-client/pypi/simple
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
import torch
import torch.utils.data as data
from python.data.dataset import *
from python.models.generator import UNet
from python.models.discriminator import PatchGAN
from python.utils.images import *
from python.train.trainer import *
from tqdm.notebook import tqdm

import sys
log = open("train.log", "a")
sys.stdout = log

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset = "/datasets/coco"
version = "2014"

In [10]:
dataset_train = CocoLab(dataset, version=version, size=256, train=True)
trainloader = data.DataLoader(dataset_train, batch_size=4, shuffle=True, num_workers=4)

dataset_test = CocoLab(dataset, version=version, size=256, train=False)
testloader = data.DataLoader(dataset_test, batch_size=4, shuffle=True, num_workers=4)

generator = UNet(1, 2).to(device)
discriminator = PatchGAN(3).to(device)

In [13]:
num_epochs = 20

In [14]:
generator.train()
discriminator.train()

gan_train = GanTrain(generator, discriminator)

train_g_avg_loss = []
train_d_avg_loss = []
test_g_avg_loss = []
test_d_avg_loss = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(num_epochs):
    train_g_losses = []
    train_d_losses = []
    test_g_losses = []
    test_d_losses = []
    
    with tqdm(trainloader, unit="batch") as tepoch:
        for L, ab in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            L = L.to(device)
            ab = ab.to(device)

            d_loss, g_loss = gan_train.step(L, ab)

            train_g_losses.append(g_loss.detach())
            train_d_losses.append(d_loss.detach())        

        with torch.no_grad():   
            # Do not set .eval()
            for L, ab in testloader:
                L = L.to(device)
                ab = ab.to(device)
                fake_ab = generator(L)
                g_loss = gan_train.generator_loss(L, ab, fake_ab)
                d_loss = gan_train.discriminator_loss(L, ab, fake_ab)
                
                test_g_losses.append(g_loss.detach())
                test_d_losses.append(d_loss.detach())

            train_g_avg_loss.append(torch.mean(torch.Tensor(train_g_losses)))
            train_d_avg_loss.append(torch.mean(torch.Tensor(train_d_losses)))
            test_g_avg_loss.append(torch.mean(torch.Tensor(test_g_losses)))
            test_d_avg_loss.append(torch.mean(torch.Tensor(test_d_losses)))

            print('[Epoch {}/{}] '.format(epoch+1, num_epochs) + "\n--- Generator ---\n" +
                    '\ttrain_loss: {:.4f} - '.format(train_g_avg_loss[-1]) +
                    'test_loss: {:.4f} - '.format(test_g_avg_loss[-1]) + "\n--- Discriminator ---\n" +
                    '\ttrain_loss: {:.4f} - '.format(train_d_avg_loss[-1]) +
                    'test_loss: {:.4f}'.format(test_d_avg_loss[-1]))

            multi_plot(testloader, generator, "figures/epoch_{}.png".format(epoch+1))
            torch.save(generator.state_dict(), "saved_models/generator")
        
plt.figure(figsize=(16, 6))
plt.title('Generator losses')
plt.plot(train_g_avg_loss)
plt.plot(test_g_avg_loss)
plt.grid()
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig("figures/generator_losses.png")


plt.figure(figsize=(16, 6))
plt.title('Discriminator losses')
plt.plot(train_d_avg_loss)
plt.plot(test_d_avg_loss)
plt.grid()
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig("figures/discriminator_losses.png")

  0%|          | 0/252 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [ ]:
LEARNING_RATE = 0.0002

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

train_avg_loss = []
test_avg_loss = []

for epoch in range(num_epochs):
    train_losses = []
    test_losses = []
    
    generator.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for L, ab in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            L = L.to(device)
            ab = ab.to(device)

            pred = generator(L)
            loss = criterion(pred, ab)

            train_losses.append(loss.detach())
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        with torch.no_grad():   
            generator.eval()

            for L, ab in testloader:
                L = L.to(device)
                ab = ab.to(device)

                pred = generator(L)
                loss = criterion(pred, ab)
                test_losses.append(loss.detach())

            train_avg_loss.append(torch.mean(torch.Tensor(train_losses)))
            test_avg_loss.append(torch.mean(torch.Tensor(test_losses)))

            print('[Epoch {}/{}] '.format(epoch+1, num_epochs) +
                    'train_loss: {:.4f} - '.format(train_avg_loss[-1]) +
                    'test_loss: {:.4f}'.format(test_avg_loss[-1]))


            generator.eval()
            ab_pred = generator(L_base).detach()
            Lab_pred = torch.concat((L_base, ab_pred), 1).to("cpu")
            tensor_to_pil(Lab_pred)[0].save("epoch_{}.png".format(epoch))
            torch.save(generator.state_dict(), "saved_models/generator")


In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.title('Losses')
plt.plot(train_avg_loss)
plt.plot(test_avg_loss)
plt.grid()
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss (L1)')


plt.show()

In [ ]:
# trying the prediction of the Unet after training
generator.eval()
ab_pred_train = generator(L).detach()
Lab_pred_train = torch.concat((L, ab_pred_train), 1).to("cpu")

In [ ]:
tensor_to_pil(Lab_pred_train)[0]